In [ ]:
import paho.mqtt.client as mqtt
import time
import aioblescan as aiobs
from aioblescan.plugins import EddyStone
import asyncio
from urllib.parse import urlparse
from datetime import datetime
import csv
import pandas as pd
from datetime import datetime
import os.path
import numpy as np
import sklearn.datasets as dt
import pandas as pd
from sklearn.preprocessing import StandardScaler
from bluezero import microbit
import serial

broker_address = "192.168.4.14" #enter your broker address here
subscribetopic = "testTopic1"
publishtopic = "testTopic2"

# (xy) is the (training_set,target) pair
def stochastic_gradient_descent(max_epochs,threshold,w_init,
                                obj_func,grad_func,xy,
                                learning_rate=0.05,momentum=0.8):
    (x_train,y_train) = xy
    w = w_init
    w_history = w
    f_history = obj_func(w,xy)
    delta_w = np.zeros(w.shape)
    i = 0
    diff = 1.0e10
    rows = x_train.shape[0]
     # Run epochs
    while  i<max_epochs and diff>threshold:
        # Shuffle rows using a fixed seed to reproduce the results
        np.random.seed(i)
        p = np.random.permutation(rows)
        
        # Run for each instance/example in training set
        for x,y in zip(x_train[p,:],y_train[p]):
            delta_w = -learning_rate*grad_func(w,(np.array([x]),y)) + momentum*delta_w
            w = w+delta_w
            
        i+=1
        w_history = np.vstack((w_history,w))
        f_history = np.vstack((f_history,obj_func(w,xy)))
        diff = np.absolute(f_history[-1]-f_history[-2])
        
    return w_history,f_history

# Input argument is weight and a tuple (train_data, target)
def grad_mse(w,xy):
    (x,y) = xy
    (rows,cols) = x.shape
    
    # Compute the output
    o = np.sum(x*w,axis=1)
    diff = y-o
    diff = diff.reshape((rows,1))    
    diff = np.tile(diff, (1, cols))
    grad = diff*x
    grad = -np.sum(grad,axis=0)
    return grad

# Input argument is weight and a tuple (train_data, target)
def mse(w,xy):
    (x,y) = xy
    
    # Compute output
    # keep in mind that wer're using mse and not mse/m
    # because it would be relevant to the end result
    o = np.sum(x*w,axis=1)
    mse = np.sum((y-o)*(y-o))
    mse = mse/2
    return mse

# Returns error rate of classifier
# total miclassifications/total*100
def error(w,xy):
    (x,y) = xy
    o = np.sum(x*w,axis=1)
    
    #map the output values to 0/1 class labels
    ind_1 = np.where(o>0.5)
    ind_0 = np.where(o<=0.5)
    o[ind_1] = 1
    o[ind_0] = 0
    return np.sum((o-y)*(o-y))/y.size*100

def get_weights(csv_file):
    df = pd.read_csv(csv_file)

    X = df[["max", "min", "humidity"]].values
    scaler = StandardScaler()


    y = df["pred_steps"].values
    y = y.reshape(y.shape[0],1)

    ones_x = np.ones((len(df), 1))
    X_ = np.concatenate((ones_x, X), axis = 1)

    y = np.asfarray(y, float)

    rand = np.random.RandomState(19)
    w_init = rand.uniform(-1,1,X_.shape[1])*.001
    w_history_stoch,mse_history_stoch = stochastic_gradient_descent(
                                    1000,0.1,w_init,
                                mse,grad_mse,(X_,y),
                                learning_rate=1e-6,momentum=0.7)
    return tuple(w_history_stoch[1000])

def print_daily_steps(csv_file):
    df = pd.read_csv(csv_file)
    (w0,w1,w2,w3) = get_weights(csv_file)
    for index, row in df.iterrows():
       row_min = row['min']
       row_max = row['max']
       row_humidity  =  row['humidity']
       new_model_steps = round(w0  + (w1* float(row_max)) + (w2* float(row_min)) + (w3*float(row_humidity)))
       old_model_steps = round(9.36932201  + (65.73209881 * float(row_max)) - (36.397804* float(row_min)) - (10.91733609*float(row_humidity)))
       print('Day {} --> original model {} --> new model {} '.format(index+1,old_model_steps,new_model_steps))

def on_connect(client, userdata, flags, rc):
#    print("Connected with result".format(str(rc)))
    client.subscribe(subscribetopic)
   
def on_message (client,userdata,msg):
    print("Message received" + msg.topic+" "+str(msg.payload))
    weather_data = msg.payload.decode()
    if weather_data:
        #today's data
        print(weather_data)
        current_date=weather_data[14:24]
        current_min_temp=weather_data[25:31]
        current_max_temp=weather_data[32:38]
        current_humidity=weather_data[39:41]

        #forecasted data
        forecast_date=weather_data[60:70]
        forecast_min_temp=weather_data[71:77]
        forecast_max_temp=weather_data[78:84]
        forecast_humidity=weather_data[85:87]

        # calculate daily weights
        
        (w0,w1,w2,w3) = get_weights("weatherdata.csv")

 
        # calculate predicted steps from daily weights
        model_steps_today = round(w0  + (w1* float(current_max_temp)) + (w2* float(current_min_temp)) + (w3*float(current_humidity)))

        model_steps_tommorrow = round(w0  + (w1* float(forecast_max_temp)) + (w2* float(forecast_min_temp)) + (w3*float(forecast_humidity)))

        # calculate predicted steps

        pred_steps_today = round(9.36932201  + (65.73209881 * float(current_max_temp)) - (36.397804* float(current_min_temp)) - (10.91733609*float(current_humidity)))
        
        pred_steps_tommorrow = round(9.36932201  + (65.73209881 * float(forecast_max_temp)) - (36.397804* float(forecast_min_temp)) - (10.91733609*float(forecast_humidity)))     

        print('!!!!!!!!!!!!!!!!!!!!       STEP 1 results       !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print('NEW MODEL says you should have walked {} steps today'.format(model_steps_today))

        print('NEW MODEL says your goal for tommorrow is {} steps'.format(model_steps_tommorrow))   
        
        print('OLD MODEL says you should have walked {} steps today'.format(pred_steps_today))

        print('OLD MODEL says your goal for tommorrow is {} steps'.format(pred_steps_tommorrow))
        print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')

        print('@@@@@@@@@@@@@@@@@@@@      STEP 2 results       @@@@@@@@@@@@@@@@@@@@@@@@@@@
        print_daily_steps("weatherdata.csv")
        print('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')
     
        # Define data
        data = [
        (current_date, current_min_temp, current_max_temp, current_humidity,model_steps_today)  ]  
        
        file_exists = os.path.isfile("/home/pi/weatherdata.csv")
        # Write CSV file
        with open("weatherdata.csv", mode='a') as fp:
            writer = csv.writer(fp, delimiter=",")
            if not file_exists:
                writer.writerow(["date","max", "min", "humidity","pred_steps"]) # write header
            writer.writerows(data)
 
def _process_packet(data):
    ev = aiobs.HCI_Event()
    xx = ev.decode(data)
    xx = EddyStone().decode(ev)
    broker_address = "192.168.4.14" #enter your broker address here
    subscribetopic = "testTopic1"
    publishtopic = "testTopic2"
    publishmessage = ""
   
    if xx:
        print("Google beacon:{}".format(xx))
        group = xx.get('url')[8:12]
        if group == 'grp4':
          steps =xx.get('url')[24:]
         
          print('***beacon received******')
          print('Step count{}'.format(steps))

          client = mqtt.Client("P1")
          client.on_connect=on_connect
          client.on_message=on_message
      
   
          if os.path.isfile("/home/pi/weatherdata.csv") and int(steps) > 0:
                weatherdata = pd.read_csv ('weatherdata.csv')
         
                if weatherdata.iloc[-1]['pred_steps'] <= int(steps):
                   publishmessage = "Goal achieved, keep going!"
                else: 
                   publishmessage = "Work harder"
                
                s = serial.Serial(timeout=0.1) 
                s.baudrate = 115200 
                s.port = '/dev/ttyACM0'
                #ttyACM0'  
                s.open() 
                while True:
                    data = publishmessage
                    if data: 
                      #print(publishmessage)
                      s.write(data.encode('utf-8'))
                      break
                    else:
                      print('No data')
                      break    
                s.close()

          print('***MQTT call success*****')
          print('*****Connecting to broker****')
          client.connect(broker_address)
          print("******Publish*****")
          client.publish(publishtopic, publishmessage)
          client.loop_start()
          print("******Subscribe****")
          ##client.subscribe(subscribetopic)
          time.sleep(10)
          client.loop_stop()

                     
if __name__=='__main__':
   
    mydev = 0
    event_loop = asyncio.get_event_loop()
    print(event_loop)
    mysocket = aiobs.create_bt_socket(mydev)
    fac = event_loop._create_connection_transport(mysocket,aiobs.BLEScanRequester,None,None)
    conn, btctrl = event_loop.run_until_complete(fac)
    btctrl.process = _process_packet
    # print('after steps display')
    print(btctrl.process)
    btctrl.send_scan_request()
    # print('after send scan')
   
    try:
        event_loop.run_forever()
        print("loop forever")
    except KeyboardInterrupt:
        print('keyboard interrupt')
    finally:
        print('closing event loop')
        btctrl.stop_scan_request()
        conn.close()
        event_loop.close()
       






